# imports

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import math
import numpy as np 
from colorama import Fore, Style, init;
import optuna

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Model and Utiliz
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score , train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import *
from sklearn.metrics import *

# Remove Warnings
import warnings 
warnings.filterwarnings("ignore")

import numpy as np
from sklearn.decomposition import PCA
from scipy.spatial.distance import mahalanobis

# Set the option to display all columns
pd.set_option('display.max_columns', None)

# Import The AUTO_ML
from flaml import AutoML

# utils

In [ ]:
palette = ['#FFFF00','#2DA8D8']

# Intlize Colors
HEAD = '#FFFF00'
TEXT = '#2DA8D8'

# Text Styling
def print_unique_header(heading,heading_color=HEAD, text_color=TEXT):
    def color_text(text, hex_color):
        # Convert hex color to RGB
        rgb = tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))
        # Apply ANSI escape code for the given RGB color
        return f"\033[38;2;{rgb[0]};{rgb[1]};{rgb[2]}m{text}\033[0m"

    bright = "\033[1m"
    reset = "\033[0m"

    total_width = len(heading) + 20
    left_space = (total_width - len(heading)) // 2
    right_space = total_width - len(heading) - left_space

    print("\n" + color_text("╭" + "─" * total_width + "╮", heading_color)) 
    print(color_text(f"│{' ' * left_space}{'▲'}{' ' * right_space}", heading_color) + reset)
    print(color_text(f"│{' ' * left_space}", heading_color) + color_text(heading, text_color) + color_text(f"{' ' * right_space}│", heading_color) + reset)
    print(color_text(f"│{' ' * left_space}{'▼'}{' ' * right_space}", heading_color) + reset)
    print(color_text("╰" + "─" * total_width + "╯", heading_color))

def print_boxed_zigzag_heading(heading, heading_color=HEAD, text_color=TEXT):
    def color_text(text, hex_color):
        # Convert hex color to RGB
        rgb = tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))
        # Apply ANSI escape code for the given RGB color
        return f"\033[38;2;{rgb[0]};{rgb[1]};{rgb[2]}m{text}\033[0m"

    bright = "\033[1m"
    reset = "\033[0m"

    heading_color_code = color_text("╭" + "─" * (len(heading) + 20) + "╮", heading_color)
    
    print("\n" + heading_color_code)
    words = heading.split()
    for i, word in enumerate(words):
        if i == len(words) - 1:
            print(f"{color_text(f'│ {word} │', text_color)}{reset}")
        else:
            print(f"{color_text(f'│ {word}', text_color)}{reset}", end=" ")
    
    print(color_text("╰" + "─" * (len(heading) + 20) + "╯", heading_color))
    
def prinT(text, hex_color=TEXT):
    # Convert hex color to RGB
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))
    # Apply ANSI escape code for the given RGB color
    colored_text = f"\033[38;2;{rgb[0]};{rgb[1]};{rgb[2]}m{text}\033[0m"
    print(colored_text)

print_unique_header("Setup Intilized")

# load data

In [ ]:
# Load Submission Data 
d_s = pd.read_csv('/kaggle/input/playground-series-s4e6/sample_submission.csv')
# Load test Data 
te_d = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv')
#Train Data 
tr_d = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv')

# Original Data 
O_D = pd.read_csv("/kaggle/input/playgrounds4e06originaldata/original.csv")

# Dropping Id from  Train 
tr_d.drop(columns=['id'], inplace=True)
te_d.drop(columns=['id'], inplace=True)
O_D.drop(columns=['id'], inplace=True)

# Concat 
tr_d = pd.concat(objs=[tr_d, O_D])

# Drop Irreleveant 
tr_d.drop(labels='Daytime/evening attendance\t', axis=1, inplace=True)
# Fill Null Values 
most_frequent = tr_d['Daytime/evening attendance'].mode()[0]
tr_d['Daytime/evening attendance'].fillna(most_frequent, inplace=True)

# Overview Fucntion
def print_boxed_blue_heading(heading):
    gradient = [Fore.BLUE, Fore.CYAN, Fore.GREEN, Fore.YELLOW, Fore.RED, Fore.MAGENTA]
    print("\n" + "=" * (len(heading) + 4))
    words = heading.split()
    for i, word in enumerate(words):
        if i == len(words) - 1:
            print(f"| {gradient[len(word) % len(gradient)] + word + Style.RESET_ALL} |")
        else:
            print(f"| {gradient[len(word) % len(gradient)] + word + Style.RESET_ALL}", end=" ")
    print("=" * (len(heading) + 4))
print_unique_header('Dat Loaded')

In [ ]:
def psod_outlier_detection_removal(data, n_components=2, threshold=3):

    # Step 1: Apply PCA to project the data into a lower-dimensional subspace
    pca = PCA(n_components=n_components)
    projected_data = pca.fit_transform(data)
    
    # Step 2: Calculate the mean and covariance matrix of the projected data
    mean = np.mean(projected_data, axis=0)
    cov_matrix = np.cov(projected_data, rowvar=False)
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    
    # Step 3: Compute Mahalanobis distance for each point in the projected space
    distances = np.array([mahalanobis(x, mean, inv_cov_matrix) for x in projected_data])
    
    # Step 4: Identify outliers based on the threshold
    outlier_mask = distances > threshold
    outliers = data[outlier_mask]
    cleaned_data = data[~outlier_mask]
    
    num_outliers = len(outliers)
    
    return cleaned_data, num_outliers

tr_d, num_outliers = psod_outlier_detection_removal(tr_d, n_components=2, threshold=3)

# print("Cleaned Data:\n", tr_d)
print("Outliers:\n", num_outliers)

In [ ]:
# Scale Data Train and Test 
S = StandardScaler()
tr_d[Num_C] = S.fit_transform(tr_d[Num_C])
te_d[Num_C] = S.transform(te_d[Num_C])
print_unique_header('Data Scaled')

In [ ]:
# Encode Target 
Le = LabelEncoder()
tr_d['Target'] = Le.fit_transform(tr_d['Target'])
print_unique_header('Target Encoded')

# flaml 

In [ ]:
# Initlize The autoML
autoML = AutoML()
# Fit autoML on Data
autoML.fit(X_TR, Y_TR, task="classification",metric='roc_auc_ovo',time_budget=3800*3)